# 閲覧サイトとURL
**netkeiba.com**:競走馬のデータ, 競走結果, 血統情報, トレーニング成績

https://www.netkeiba.com/



In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# データを格納するためのリストを初期化
horse_data_list = []

# page=1からpage=626までの各ページを順にスクレイピング
for page in range(1, 5):
    # スクレイピングするウェブページのURL
    url = f'https://db.netkeiba.com//?pid=horse_list&word=&x=27&y=6&act=1&page={page}'
    # requestsを使ってURLの内容を取得
    response = requests.get(url)
    # ステータスコードが200（正常にコンテンツが取得できた）場合のみ処理を続行
    if response.status_code == 200:
        # BeautifulSoupを使ってHTMLを解析
        soup = BeautifulSoup(response.content, 'html.parser')

        # 各馬の情報が含まれる<td>要素を取得
        horse_cells = soup.find_all('td', class_='bml txt_l')

        # 各<td>要素から情報を抽出してリストに追加
        for cell in horse_cells:
            horse_name = cell.find('a').text.strip()
    
            sex = cell.find_next_sibling(class_='txt_c').text.strip()
    
            birth_year_element = cell.find_next_sibling(class_='txt_c').find_next_sibling(class_='txt_c').find('a')
            birth_year = birth_year_element.text.strip() if birth_year_element else ''
    
            trainer_element = cell.find_next_sibling(class_='txt_l').find('a')
            trainer = trainer_element.text.strip() if trainer_element else ''
    
            sire_element = cell.find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find('a')
            sire = sire_element.text.strip() if sire_element else ''
    
            mare_element = cell.find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find('a')
            mare = mare_element.text.strip() if mare_element else ''
    
            damsire_element = cell.find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find('a')
            damsire = damsire_element.text.strip() if damsire_element else ''
    
            owner_element = cell.find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find('a')
            owner = owner_element.text.strip() if owner_element else ''
    
            breeder_element = cell.find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find_next_sibling(class_='txt_l').find('a')
            breeder = breeder_element.text.strip() if breeder_element else ''
    
            prize_money = cell.find_next_sibling(class_='txt_r').text.strip()

            horse_data_list.append([horse_name, sex, birth_year, trainer, sire, mare, damsire, owner, breeder, prize_money])

        # 馬の情報がリストに格納されました
        for horse_data in horse_data_list:
            

        # 少し待機してから次のページへ（ウェブサイトへの負荷軽減のため）
            time.sleep(1)

# データをCSVファイルに書き込む
with open('horse_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    
    # CSVのヘッダーを書き込む
    writer.writerow(['馬名', '性別', '生年', '厩舎', '父', '母', '母父', '馬主', '生産者', '総賞金(万円)'])
    
    # 馬の情報をCSVに書き込む
    for horse_data in horse_data_list:
        writer.writerow(horse_data)

print('CSVファイルにデータを書き込みました。')


CSVファイルにデータを書き込みました。


In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# データを格納するためのリストを初期化
horse_data_list = []

# page=1からpage=4までの各ページを順にスクレイピング
for page in range(1, 2):
    url = f'https://db.netkeiba.com//?pid=horse_list&word=&x=27&y=6&act=1&page={page}'
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        horse_cells = soup.find_all('td', class_='bml txt_l')

        for cell in horse_cells:
            horse_name = cell.find('a').text.strip()
            horse_url = 'https://db.netkeiba.com' + cell.find('a')['href']  # 馬の詳細ページのURL
            
            # 馬の詳細ページにアクセスしてゴールタイムを取得
            horse_response = requests.get(horse_url)
            horse_soup = BeautifulSoup(horse_response.content, 'html.parser')
            goal_time_tag = horse_soup.find('td', string='ゴールタイム')  
            goal_time = goal_time_tag.find_next('td').text.strip() if goal_time_tag else 'データなし'
            
            # 馬のデータをリストに追加
            horse_data_list.append([horse_name, goal_time])
            time.sleep(1)

# データをCSVファイルに書き込む
with open('horse_goal_times.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['馬名', 'ゴールタイム'])
    for horse_data in horse_data_list:
        writer.writerow(horse_data)

print('CSVファイルにゴールタイムを書き込みました。')


CSVファイルにゴールタイムを書き込みました。


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def scrape_race_data(url):
    # レスポンスの取得
    response = requests.get(url)
    if response.status_code != 200:
        return "Error: Unable to fetch the page."

    # データを解析
    soup = BeautifulSoup(response.content, 'html.parser')

    # データを格納するリストを初期化
    race_data_list = []

    # レース情報の取得
    race_info = soup.find('div', class_='RaceData01').get_text(strip=True)
    distance = race_info.split('/')[0].split('m')[0]  # 距離の抽出

    # 各馬の情報を取得
    horse_rows = soup.find_all('tr', class_='HorseList')
    for row in horse_rows:
        cells = row.find_all('td')
        if cells:
            horse_name = cells[3].find('a').get_text(strip=True)  # 馬名
            horse_weight = cells[14].get_text(strip=True).split('(')[0]  # 馬体重
            goal_time = cells[11].get_text(strip=True)  # ゴールタイム
            race_data_list.append([horse_name, horse_weight, goal_time, distance])

    return race_data_list

# メインの実行部
if __name__ == '__main__':
    # スクレイピングするレースのURL
    race_url = 'https://www.example.com/race/result/url'
    race_results = scrape_race_data(race_url)

    # CSVに書き込む
    with open('race_data.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['馬名', '馬体重', 'ゴールタイム', '距離'])
        writer.writerows(race_results)

    print('データがCSVファイルに保存されました。')
